In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.4.0a0+git74cd574


In [2]:
import os

# Use the eager mode
os.environ['PT_HPU_LAZY_MODE'] = '0'

# Verify the environment variable is set
print(f"PT_HPU_LAZY_MODE: {os.environ['PT_HPU_LAZY_MODE']}")

import habana_frameworks.torch.core as htcore


PT_HPU_LAZY_MODE: 0


Calling add_step_closure function does not have any effect. It's lazy mode only functionality. (warning logged once)
Calling mark_step function does not have any effect. It's lazy mode only functionality. (warning logged once)
Calling iter_mark_step function does not have any effect. It's lazy mode only functionality. (warning logged once)
/usr/lib/python3.10/inspect.py:288: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(object, types.FunctionType)


In [3]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

# Tutorial 6  
Graph AutoEncoders GAE &  
Variational Graph Autoencoders VGAE    

[paper](https://arxiv.org/pdf/1611.07308.pdf)  
[code](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/autoencoder.py)

## Graph AutoEncoder GAE

### Load the data

In [4]:
dataset = Planetoid("data", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Processing...
Done!
/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [5]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [6]:
data = train_test_split_edges(data)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [7]:
data

Data(x=[3327, 3703], y=[3327], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

### Define the Encoder

In [8]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    

### Define the Autoencoder

In [9]:
from torch_geometric.nn import GAE

In [10]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device("hpu")
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

============================= HABANA PT BRIDGE CONFIGURATION =========================== 
 PT_HPU_LAZY_MODE = 0
 PT_RECIPE_CACHE_PATH = 
 PT_CACHE_FOLDER_DELETE = 0
 PT_HPU_RECIPE_CACHE_CONFIG = 
 PT_HPU_MAX_COMPOUND_OP_SIZE = 9223372036854775807
 PT_HPU_LAZY_ACC_PAR_MODE = 1
 PT_HPU_ENABLE_REFINE_DYNAMIC_SHAPES = 0
 PT_HPU_EAGER_PIPELINE_ENABLE = 1
 PT_HPU_EAGER_COLLECTIVE_PIPELINE_ENABLE = 1
---------------------------: System Configuration :---------------------------
Num CPU Cores : 160
CPU RAM       : 2113407800 KB
------------------------------------------------------------------------------


In [12]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)



In [13]:
model.train()
model = torch.compile(model, backend="hpu_backend")

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.6150, AP: 0.6365
Epoch: 002, AUC: 0.6266, AP: 0.6473
Epoch: 003, AUC: 0.6295, AP: 0.6512
Epoch: 004, AUC: 0.6271, AP: 0.6491
Epoch: 005, AUC: 0.6279, AP: 0.6498
Epoch: 006, AUC: 0.6285, AP: 0.6506
Epoch: 007, AUC: 0.6292, AP: 0.6512
Epoch: 008, AUC: 0.6294, AP: 0.6517
Epoch: 009, AUC: 0.6299, AP: 0.6524
Epoch: 010, AUC: 0.6306, AP: 0.6535
Epoch: 011, AUC: 0.6317, AP: 0.6554
Epoch: 012, AUC: 0.6338, AP: 0.6586
Epoch: 013, AUC: 0.6357, AP: 0.6621
Epoch: 014, AUC: 0.6377, AP: 0.6663
Epoch: 015, AUC: 0.6398, AP: 0.6708
Epoch: 016, AUC: 0.6419, AP: 0.6758
Epoch: 017, AUC: 0.6437, AP: 0.6814
Epoch: 018, AUC: 0.6453, AP: 0.6870
Epoch: 019, AUC: 0.6466, AP: 0.6920
Epoch: 020, AUC: 0.6470, AP: 0.6963
Epoch: 021, AUC: 0.6484, AP: 0.7008
Epoch: 022, AUC: 0.6485, AP: 0.7043
Epoch: 023, AUC: 0.6492, AP: 0.7079
Epoch: 024, AUC: 0.6500, AP: 0.7109
Epoch: 025, AUC: 0.6504, AP: 0.7133
Epoch: 026, AUC: 0.6508, AP: 0.7154
Epoch: 027, AUC: 0.6508, AP: 0.7164
Epoch: 028, AUC: 0.6511, AP:

In [25]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[ 0.1883, -0.3782],
        [-0.2293,  1.1419],
        [ 0.1883, -0.3782],
        ...,
        [-0.0636,  0.6200],
        [ 0.1883, -0.3782],
        [ 0.1883, -0.3782]], device='hpu:0', grad_fn=<AddBackward0>)

## Are the results (AUC) and (AP) easy to read and compare?

# Use Tensorboard

In [26]:
from torch.utils.tensorboard import SummaryWriter

In [27]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('hpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Import tensorboard

#### Installation: (if needed) "pip install tensorboard"

In [28]:
writer = SummaryWriter('runs/GAE1_experiment_'+'2d_100_epochs')

In [29]:
model.train()
model = torch.compile(model, backend="hpu_backend")

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6138, AP: 0.6300
Epoch: 002, AUC: 0.6327, AP: 0.6525
Epoch: 003, AUC: 0.6329, AP: 0.6523
Epoch: 004, AUC: 0.6336, AP: 0.6534
Epoch: 005, AUC: 0.6335, AP: 0.6539
Epoch: 006, AUC: 0.6336, AP: 0.6543
Epoch: 007, AUC: 0.6332, AP: 0.6544
Epoch: 008, AUC: 0.6334, AP: 0.6546
Epoch: 009, AUC: 0.6332, AP: 0.6546
Epoch: 010, AUC: 0.6330, AP: 0.6547
Epoch: 011, AUC: 0.6329, AP: 0.6549
Epoch: 012, AUC: 0.6327, AP: 0.6551
Epoch: 013, AUC: 0.6333, AP: 0.6560
Epoch: 014, AUC: 0.6342, AP: 0.6578
Epoch: 015, AUC: 0.6353, AP: 0.6602
Epoch: 016, AUC: 0.6368, AP: 0.6634
Epoch: 017, AUC: 0.6383, AP: 0.6671
Epoch: 018, AUC: 0.6400, AP: 0.6710
Epoch: 019, AUC: 0.6418, AP: 0.6753
Epoch: 020, AUC: 0.6432, AP: 0.6797
Epoch: 021, AUC: 0.6439, AP: 0.6838
Epoch: 022, AUC: 0.6450, AP: 0.6882
Epoch: 023, AUC: 0.6454, AP: 0.6921
Epoch: 024, AUC: 0.6460, AP: 0.6956
Epoch: 025, AUC: 0.6460, AP: 0.6987
Epoch: 026, AUC: 0.6465, AP: 0.7016
Epoch: 027, AUC: 0.6469, AP: 0.7045
Epoch: 028, AUC: 0.6472, AP:

## Graph Variational AutoEncoder (GVAE)

In [30]:
from torch_geometric.nn import VGAE

In [31]:
dataset = Planetoid("data", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)


class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [32]:
out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('hpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [33]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [34]:
writer = SummaryWriter('runs/VGAE_experiment_'+'2d_100_epochs')

model.train()
model = torch.compile(model, backend="hpu_backend")
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6046, AP: 0.6447
Epoch: 002, AUC: 0.6140, AP: 0.6550
Epoch: 003, AUC: 0.6167, AP: 0.6579
Epoch: 004, AUC: 0.6183, AP: 0.6585
Epoch: 005, AUC: 0.6203, AP: 0.6594
Epoch: 006, AUC: 0.6219, AP: 0.6609
Epoch: 007, AUC: 0.6231, AP: 0.6616
Epoch: 008, AUC: 0.6241, AP: 0.6622
Epoch: 009, AUC: 0.6246, AP: 0.6623
Epoch: 010, AUC: 0.6252, AP: 0.6627
Epoch: 011, AUC: 0.6258, AP: 0.6628
Epoch: 012, AUC: 0.6262, AP: 0.6630
Epoch: 013, AUC: 0.6266, AP: 0.6635
Epoch: 014, AUC: 0.6269, AP: 0.6638
Epoch: 015, AUC: 0.6271, AP: 0.6641
Epoch: 016, AUC: 0.6275, AP: 0.6645
Epoch: 017, AUC: 0.6278, AP: 0.6649
Epoch: 018, AUC: 0.6283, AP: 0.6658
Epoch: 019, AUC: 0.6288, AP: 0.6664
Epoch: 020, AUC: 0.6290, AP: 0.6668
Epoch: 021, AUC: 0.6294, AP: 0.6674
Epoch: 022, AUC: 0.6299, AP: 0.6679
Epoch: 023, AUC: 0.6302, AP: 0.6685
Epoch: 024, AUC: 0.6309, AP: 0.6691
Epoch: 025, AUC: 0.6313, AP: 0.6697
Epoch: 026, AUC: 0.6316, AP: 0.6702
Epoch: 027, AUC: 0.6318, AP: 0.6705
Epoch: 028, AUC: 0.6319, AP: